<a href="https://colab.research.google.com/github/malienist/FIRST-JP/blob/main/notebooks/w_imdb_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adversarial Attacks on IMDB Sentiment Analysis using BERT-Attack (BAE)

In this notebook, we will explore how to perform adversarial attacks on a pre-trained sentiment analysis model using BERT-Attack (BAE). Adversarial attacks are designed to fool machine learning models by introducing small perturbations in the input data, leading to incorrect predictions. BERT-Attack leverages the power of BERT, a masked language model, to generate adversarial examples by replacing words in the input text.

## Objectives

- Load and prepare a pre-trained sentiment analysis model (`distilbert-base-uncased-finetuned-sst-2-english`).
- Utilize the IMDB dataset for sentiment analysis.
- Apply the BERT-Attack (BAE) to create adversarial examples.
- Evaluate the model's predictions on the adversarial examples.

## Steps

1. **Load the Pre-trained Model and Tokenizer**: We'll use the HuggingFace Transformers library to load the model and tokenizer.
2. **Set up TextAttack**: We'll configure TextAttack to use BERT-Attack for generating adversarial examples.
3. **Load the IMDB Dataset**: We'll use the IMDB dataset, which contains movie reviews labeled as positive or negative.
4. **Generate Adversarial Examples**: We'll apply BERT-Attack to a set of samples from the dataset and observe the changes in the model's predictions.
5. **Evaluate Results**: We'll compare the original and adversarial predictions to understand the effectiveness of the attack.

## Requirements

Ensure you have the necessary libraries installed:

```python
!pip install transformers textattack


In [ ]:
!pip install textattack torch torchvision

# Next Steps


*   Load and Prepare the Model
*   Wrap the Model for TextAttack
*   Load the IMDB dataset




In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from textattack.attack_recipes import BAEGarg2019
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset, Dataset
from textattack import Attacker

# Load a pre-trained sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a HuggingFace pipeline for sentiment analysis
pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Wrap the HuggingFace model for TextAttack
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Load the IMDB dataset
dataset = HuggingFaceDataset("imdb", split="test")


textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set 

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

textattack: Loading datasets dataset imdb, split test.


# Nest Steps

*   Create a Custom Dataset for TextAttack
*   Define the Attack Method
*   Perform the Attack
*   Evaluate and Print the Results


In [ ]:
# Select multiple samples from the dataset
num_samples = 10  # Specify the number of samples you want to attack
samples = [(dataset[i][0]['text'], dataset[i][1]) for i in range(num_samples)]

# Create a custom dataset for TextAttack
custom_dataset = Dataset(samples, input_columns=["text"])

# Define the attack method
attack = BAEGarg2019.build(model_wrapper)

# Perform the attack
attacker = Attacker(attack, custom_dataset)

# Attack each sample individually and collect results
results = attacker.attack_dataset()

# Print the adversarial text and predictions
for result in results:
    adversarial_text = result.perturbed_text()
    adversarial_result = pipe([adversarial_text])
    print(f"Adversarial text: {adversarial_text}")
    print(f"Adversarial prediction: {adversarial_result}")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

[Succeeded / Failed / Skipped / Total] 8 / 0 / 1 / 9:  90%|█████████ | 9/10 [38:51<04:19, 259.06s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[0 (100%)]] --> [[1 (94%)]]

Technically I'am a Van Damme Fan, or I was. this [[movie]] is so [[bad]] that I hated myself for wasting those 90 minutes. Do not let the name Isaac Florentine (Undisputed II) fool you, I had big hopes for this one, depending on what I saw in (Undisputed II), man.. was I wrong ??! all action fans wanted a big comeback for the classic action hero, but i guess we [[wont]] be able to see that soon, as our hero keep coming with those (going -to-a-border - far-away-town-and -kill -the-bad-guys- than-comeback- home) movies I mean for God's sake, we are in 2008, and they insist on doing those [[disappointing]] movies on every level. Why ??!!! Do your self a favor, skip it.. seriously.

Technically I'am a Van Damme Fan, or I was. this [[film]] is so [[entertaining]] that I hated myself for wasting those 90 minutes. Do not let the name Isaac Florentine (Undisputed I

[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10: 100%|██████████| 10/10 [46:04<00:00, 276.49s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[0 (100%)]] --> [[1 (51%)]]

Honestly [[awful]] film, [[bad]] editing, awful lighting, dire dialog and scrappy screenplay.<br /><br />The lighting at is so [[bad]] there's moments you can't even see what's going on, I even tried to playing with the contrast and brightness so I could see something but that didn't help.<br /><br />They must have found the [[script]] in a bin, the character development is just as [[awful]] and while you hardly expect much from a Jean-Claude Van Damme film this one manages to hit an all time low. You can't [[even]] laugh at the cheesy'ness.<br /><br />The directing and editing are also [[terrible]], the whole film follows an extremely tired routine and [[fails]] at every turn as it bumbles through the plot that is so weak it's just unreal.<br /><br />There's not a lot else to say other than it's [[really]] bad and nothing like Jean-Claude Van Damme's earl


Adversarial text: we love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I liked to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). hollywood prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't disrupt the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' look. (I'm sure there are those of you out there who think Babylon 5 is worth sci-fi TV. It's not. It's clichéd and funny.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself lightly (cf. Star Trek). It will treat important issues, yet not as a serious philosophy. It's really good to care about the characters here as they are not simply foolish, just showing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they h